In [6]:
!pip install pandas

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\sangdal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [11]:
!pip install requests

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\sangdal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [153]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup #파싱 할때 쓰이는 모듈

In [137]:
driver = webdriver.Chrome()
driver.get('https://www.banapresso.com/store')
soup = BeautifulSoup(driver.page_source)
# print(soup)

In [66]:
#  i태그에서 이름 파싱하기 
store_name_tag = soup.findAll('i')
print(store_name_tag)

[<i>가산디지털단지역점</i>, <i>강남구청점</i>, <i>강남역사거리점</i>, <i>강남역점</i>, <i>강남점</i>, <i>강남효성점</i>, <i>강동역점</i>, <i>거여역점</i>, <i>건대역점</i>, <i>경희대점</i>]


In [55]:
# 매장명 파싱해서 텍스트로 나타내기 
for i in range(len(store_name_tag)):
    store_name = store_name_tag[i].text
    print(store_name)

가산디지털단지역점
강남구청점
강남역사거리점
강남역점
강남점
강남효성점
강동역점
거여역점
건대역점
경희대점


In [73]:
# 주소 파싱하여 텍스트로 나타내기 
address = soup.findAll('span',{'class','store_name_map'})

for span in address :
    inner_span = span.findAll('span')
    print(inner_span[0].text)

서울시 금천구 가산동 60-3 
서울 강남구 청담동 45-4
서울특별시 강남구 역삼동 820-10
서울 강남구 역삼동822-7
서울 강남구 테헤란로4길 46 (역삼동 826-37) 쌍용 플래티넘, 1층
서울 강남구 역삼동 825-24
서울 강동구 천호동 447
서울특별시 송파구 거여동 37-6
서울 광진구 아차산로 221 (화양동 7-4)
서울 동대문구 경희대로4길 1


In [139]:
# 코드 정리하기 
store_info = soup.findAll('span', {'class', 'store_name_map'})

for i in range(len(store_info)):
    store_name_tag = store_info[i].findAll('i')
    inner_span = store_info[i].findAll('span')
    print('{} - {}'.format(store_name_tag[0].text.strip(), inner_span[0].text.strip()))

가산디지털단지역점 - 서울시 금천구 가산동 60-3
강남구청점 - 서울 강남구 청담동 45-4
강남역사거리점 - 서울특별시 강남구 역삼동 820-10
강남역점 - 서울 강남구 역삼동822-7
강남점 - 서울 강남구 테헤란로4길 46 (역삼동 826-37) 쌍용 플래티넘, 1층
강남효성점 - 서울 강남구 역삼동 825-24
강동역점 - 서울 강동구 천호동 447
거여역점 - 서울특별시 송파구 거여동 37-6
건대역점 - 서울 광진구 아차산로 221 (화양동 7-4)
경희대점 - 서울 동대문구 경희대로4길 1


In [148]:
# 1페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[1]/a
# 2페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[2]/a
# 3페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[3]/a
# 4페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[4]/a
# 5페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[5]/a
# 6페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[1]/a
# 11페이지 /html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li/a

driver = webdriver.Chrome()
driver.get('https://www.banapresso.com/store')
page = 0

data = []
    
while True:
    
    if(page <= 1):
        #1~5 돌리기 
        for i in range(1, 6):
            
            innerpage_xpath = f'/html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li[{i}]/a'
            driver.find_element(By.XPATH, innerpage_xpath).click()
            print(f'*****{page+1}페이지 - {i}*****\n')
            
            soup = BeautifulSoup(driver.page_source)
            store_info = soup.findAll('span', {'class', 'store_name_map'})

            for i in range(len(store_info)):
                store_name= store_info[i].findAll('i')
                store_address = store_info[i].findAll('span')
                print('{} - {}'.format(store_name[0].text.strip(), store_address[0].text.strip()))
                data.append({'Store Name': store_name[0].text.strip(), 'Address': store_address[0].text.strip()})
        
        # 6번을 보기 위해 다음페이지로 넘기기 
        page_xpath = '/html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/span[2]/a'
        driver.find_element(By.XPATH, page_xpath).click()
        # 마지막 11번 페이지 안넘기게 페이지 변수를 1씩 증가시킴
        page += 1
        
    else:
        
        innerpage_xpath = f'/html/body/div/div/div/div/article/div/section/div/div[1]/div[3]/ul/li/a'
        driver.find_element(By.XPATH, innerpage_xpath).click()
        print(f'*****마지막페이지*****\n')

        soup = BeautifulSoup(driver.page_source)
        store_info = soup.findAll('span', {'class', 'store_name_map'})

        for i in range(len(store_info)):
            store_name = store_info[i].findAll('i')
            store_address = store_info[i].findAll('span')
            print('{} - {}'.format(store_name[0].text.strip(), store_address[0].text.strip()))
            data.append({'Store Name': store_name[0].text.strip(), 'Address': store_address[0].text.strip()})
            
        break

*****1페이지 - 1*****

가산디지털단지역점 - 서울시 금천구 가산동 60-3
강남구청점 - 서울 강남구 청담동 45-4
강남역사거리점 - 서울특별시 강남구 역삼동 820-10
강남역점 - 서울 강남구 역삼동822-7
강남점 - 서울 강남구 테헤란로4길 46 (역삼동 826-37) 쌍용 플래티넘, 1층
강남효성점 - 서울 강남구 역삼동 825-24
강동역점 - 서울 강동구 천호동 447
거여역점 - 서울특별시 송파구 거여동 37-6
건대역점 - 서울 광진구 아차산로 221 (화양동 7-4)
경희대점 - 서울 동대문구 경희대로4길 1
*****1페이지 - 2*****

광명하안점 - 광명시 하안동 36-2
교대사거리점 - 서울 서초구 서초동 1598-1
교대역점 - 서울 서초구 반포대로30길 82
교대점 - 서울 서초구 서초대로54길 27, 1층
교육개발원사거리점 - 서울 서초구 강남대로27길 7-21 1층
구로비즈메트로점 - 서울시 구로구 구로동 851번지 한화비즈메트로1차 109호
구로지플러스점 - 서울시 구로구 구로동 222-31
국기원사거리점 - 서울 강남구 테헤란로8길 26 (역삼동827-48)
군포첨단산업단지점 - 경기도 군포시 부곡동 1244-4 1층
길동역점 - 서울 강동구 양재대로 1490 (길동 366-5)
*****1페이지 - 3*****

김포구래점 - 경기도 김포시 구래동 6883-10 103호
낙성대역점 - 서울시 관악구 봉천동 1658-19
남대문점 - 서울 중구 세종대로 15 남대문5가 6-7
남부터미널점 - 서울 서초구 효령로57길 1
답십리역점 - 서울특별시 동대문구 천호대로 289, 한일노벨리아 104호
대치선릉점 - 서울특별시 강남구 대치동 890-45
대치역점 - 서울 강남구 대치동 507
대치은마점 - 서울 강남구 도곡로 451 (대치동 941-22) 한나빌딩, 1층
대치점 - 서울 강남구 테헤란로 84길 14 (대치동 942-19), 1층
대치포스코점 - 서울 강남구 테헤란로78길 16 (대치동 891-44) 노

In [150]:
print(data)

[{'Store Name': '가산디지털단지역점', 'Address': '서울시 금천구 가산동 60-3'}, {'Store Name': '강남구청점', 'Address': '서울 강남구 청담동 45-4'}, {'Store Name': '강남역사거리점', 'Address': '서울특별시 강남구 역삼동 820-10'}, {'Store Name': '강남역점', 'Address': '서울 강남구 역삼동822-7'}, {'Store Name': '강남점', 'Address': '서울 강남구 테헤란로4길 46 (역삼동 826-37) 쌍용 플래티넘, 1층'}, {'Store Name': '강남효성점', 'Address': '서울 강남구 역삼동 825-24'}, {'Store Name': '강동역점', 'Address': '서울 강동구 천호동 447'}, {'Store Name': '거여역점', 'Address': '서울특별시 송파구 거여동 37-6'}, {'Store Name': '건대역점', 'Address': '서울 광진구 아차산로 221 (화양동 7-4)'}, {'Store Name': '경희대점', 'Address': '서울 동대문구 경희대로4길 1'}, {'Store Name': '광명하안점', 'Address': '광명시 하안동 36-2'}, {'Store Name': '교대사거리점', 'Address': '서울 서초구 서초동 1598-1'}, {'Store Name': '교대역점', 'Address': '서울 서초구 반포대로30길 82'}, {'Store Name': '교대점', 'Address': '서울 서초구 서초대로54길 27, 1층'}, {'Store Name': '교육개발원사거리점', 'Address': '서울 서초구 강남대로27길 7-21 1층'}, {'Store Name': '구로비즈메트로점', 'Address': '서울시 구로구 구로동 851번지 한화비즈메트로1차 109호'}, {'Store Name': '구로지플러스점', 'Address': 

In [154]:
df = pd.DataFrame(data)
df

,Store Name,Address
0,가산디지털단지역점,서울시 금천구 가산동 60-3
1,강남구청점,서울 강남구 청담동 45-4
2,강남역사거리점,서울특별시 강남구 역삼동 820-10
3,강남역점,서울 강남구 역삼동822-7
4,강남점,"서울 강남구 테헤란로4길 46 (역삼동 826-37) 쌍용 플래티넘, 1층"
...,...,...
105,가산파트너스타워점,
106,구로우림1차점,
107,문정테라타워점,"서울특별시 송파구 송파대로 167, 문정역테라타워 A동 G123호"
108,시흥은계점,"경기도 시흥시 은계번영길 11,111호"


In [156]:
df.to_excel('바나프레소_매장정보.xlsx')

In [155]:
!pip install openpyxl

You should consider upgrading via the 'C:\Users\sangdal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
